## Извлечение отношений

Традиционно извлечение отношений решается как задача классификации. Нужно связать именованные сущности в предложении какими-то заранее известными типами связей. Чаще всего это отношения вроде work_at, born_in, located_in, head_of. В биомедициских текстах извлечение отношений применяется для извлечения взаимодействия белков и поиска пар (лекарство, болезнь). Количество аргументов вообще может быть любым, но чаще всего ограничиваются бинарными отношениями (субъект, предикат, объект). 

Посмотрим как это работает на размеченном датасете.

Данные - https://github.com/thunlp/FewRel/blob/master/data/train.json


FewRel - это датасет для few-shot обучения. Это немного другая задача, для которой нужна нейронная сеть с определенной архитектурой и тест сет тут состоит из отношений других типов. Подробнее про задачу и датасет можно почитать вот тут - https://arxiv.org/pdf/1810.10147v1.pdf.

Мы его будем использовать для обычной классификации. Оценивать качество будем на кросс-валидации.


In [ ]:
import json
import os
from collections import Counter
from itertools import combinations
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from scipy.sparse import hstack
from collections import defaultdict
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import spacy
nlp = spacy.load('en_core_web_sm')

Посмотрим на данные.

In [ ]:
data = json.load(open('train.json'))

В датасете 64 типа отношений и у каждого типа 700 - предложений, в которых оно встречается.

In [ ]:
len(data.keys())

Типы отношений обознчаются как-то условно, но в статье про датасет можно посмотреть соответствие.

In [ ]:
data.keys()

In [ ]:
c = Counter([len(data[k]) for k in data])

In [ ]:
c

Каждый инстанс - это предложения и разметка. Ключ h - это главное слово, t - зависимое. Предложения разделены на токены и в разметке указаны слова, уникальный номер и индексы сущности в предложении.

In [ ]:
data['P127'][0]

Для обучения нам нужно каким-то образом перевести такую разметку в один вектор и сопоставить ему тип отношения.

Как это сделать? 

Стандартный способ - достать контекст слева от первой сущности, между сущностями и после второй сущности. Левые и правые контексты можно ограничить каким-то числом (например, 3 слова). Для каждого контекста можно получить вектор обычными способами - например через TfidfVectorizer. Потом эти вектора конкатенируются в один.

Ещё в этот вектор можно добавить длину контекста, тэги сущностей, сами сущности, порядок сущностей и т.д.

In [ ]:
# тут будем держать сущности
ent1 = []
ent2 = []

# тут будем хранить контексты
left = []
right = []
middle = []

# целые предложения тоже на всякий случай достанем
sents = []

# целевая переменная (тип отношений будет тут)
target = []


# проходим по типам отношений
for key in data:
    # по каждому инстансу
    for instance in data[key]:
        
        tokens = instance['tokens']
        sents.append(tokens)
        
        ent1.append(' '.join([tokens[i] for i in instance['h'][2][0]]))
        ent2.append(' '.join([tokens[i] for i in instance['t'][2][0]]))
        
        
        # h и t не обязательно идут в таком порядке
        # чтобы достать контексты нужно понять что из них идет первым
        if instance['h'][2][0][0] < instance['t'][2][0][0]:
            first, second = 'h', 't'
        else:
            second, first = 'h', 't'
        
        
        # индексы сущностей
        first_start = instance[first][2][0][0]
        first_end = instance[first][2][0][-1]
        second_start = instance[second][2][0][0]
        second_end = instance[second][2][0][-1]

        # левый контекст - это три слова слева от начала первой сущности
        # если слева меньше 3 слов, то добавим тэгов <START>
        left_context = tokens[max(0, first_start-3):first_start]
        left.append((['<START>']*(3-len(left_context))) + left_context)
        
        # правый контекст - это 3 слова после последнего слова второй сущности
        # если справа меньше 3 слов, то добавим тэгов <END>
        right_context = tokens[second_end+1:second_end+4]
        right.append(right_context + (['<END>']*(3-len(right_context))))

        # средний контекст - это слова между последний словом первой сущности 
        # и первым словом второй сущности
        middle_context = tokens[first_end+1:second_start]
        middle.append(middle_context)

        target.append(key)
        
        

Для тфидф векторайзера склеим токены. Обучим один общий векторайзер на всех текстах, но можно для каждого контекста обучить свой. 

In [ ]:
lefts_s = [' '.join(l) for l in left]
rights_s = [' '.join(l) for l in right]
middles_s = [' '.join(l) for l in middle]

tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
tfidf.fit(lefts_s + rights_s + middles_s)

l = tfidf.fit_transform(lefts_s)
r = tfidf.fit_transform(rights_s)
m = tfidf.fit_transform(middles_s)

X = csr_matrix(hstack([l,m,r])) # чтобы можно было по индексам доставать

In [ ]:
X.shape

In [ ]:
y = np.array(target)

Оценим качество на StratifiedKFold. Посчитаем стандартные метрики (c микро и макро усреднением). Ещё сделаем матрицу ошибок.

In [ ]:
N = 4
skf = StratifiedKFold(n_splits=N, shuffle=True)
metrics_macro = np.zeros((3))
metrics_micro = np.zeros((3))
conf = np.zeros((len(set(y)), len(set(y))))

for train_index, test_index in skf.split(X, y):
    # Можно конечно что-нибудь посложнее, но для примера хватит и логрега
    clf = LogisticRegression()
    clf.fit(X[train_index], y[train_index])
    preds = clf.predict(X[test_index])
    
    metrics_macro += precision_recall_fscore_support(y[test_index], preds, average='macro')[:3]
    metrics_micro += precision_recall_fscore_support(y[test_index], preds, average='micro')[:3]
    
    conf += confusion_matrix(y[test_index], preds)
    

In [ ]:
print(metrics_micro/N)
print(metrics_macro/N)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(data=np.round(conf/3).astype(int), 
            annot=True, 
            fmt="d", xticklabels=clf.classes_, yticklabels=clf.classes_, ax=ax)
plt.title("Confusion matrix")
plt.show()

Качество получается не очень хорошее. Но для такой сложной задачи (и не очень подходящего датасета) - это нормально.

Это качество ещё сильнее ухудшится, если мы захотим применить классификатор на неразмеченных текстах. В этом датасете нам сразу даны разметки именованных сущностей. В реальной задаче - их нужно находить самим.

Посмотрим как это работает на каких-нибудь новостных текстах. Именованные сущности достанем с помощью spacy.

Новостные тексты взяты отсюда - https://webhose.io/datasets/

In [ ]:
files = os.listdir('news')

In [ ]:
doc = json.load(open('news/'+files[0]))

Тут полно метаинформации, но нам нужны только тексты.

In [ ]:
doc

In [ ]:
doc = nlp(doc['text'])

Spacy сразу достает сущности.

In [ ]:
for ent in doc.ents:
    print(ent.string, ent.label_)

Тут много разных типов сущностей. Возьмем только PERSON и ORG (GPE тоже скорее организация, поэтому включим её тоже).

Нас интересуют только предложения, в которых есть хотя бы одна персона и одна организация.

In [ ]:
sents_with_ents = []

for file in files:
    js = json.load(open('news/'+file))
    text = js['text']
    doc = nlp(text)
    
    
    for sent in doc.sents:
        sent = sent.as_doc()
        pers = []
        orgs = []
        for entity in sent.ents:
            if entity.label_ == 'PERSON':
                pers.append(entity)
            elif entity.label_ in ['ORG', 'GPE']:
                orgs.append(entity)
        if pers and orgs:
            sents_with_ents.append((sent, pers, orgs))
                
                


Теперь точно также извлечем контексты вокруг сущностей. Единственное отличие - нам нужно учитывать случаи, когда в одном предложении сразу несколько сущностей. Просто переберем все сочетания персон с организациями. 

In [ ]:
lefts = []
ents1 = []
ents2 = []
rights = []
middles = []
orders = []
sents = []
tags = []
for sent, pers, orgs in sents_with_ents:
    for per in pers:
        # пары с этой персоной и остальными организациями
        pairs = [[per, org] for org in orgs]
        
        sent = list(sent)
        
        for ent1, ent2 in pairs:
            sents.append(' '.join([str(w) for w in sent]))
            if ent1.start > ent2.start:
                ent2, ent1 = ent1, ent2

            # ent.start в spacy совпадает с начальным токеном
            # а вот ent.end - это номер последнего токена + 1
            # поэтому этот кусок немного отличается
            left_context = sent[max(0, ent1.start-3):ent1.start]
            lefts.append(['<START>']*(3-len(left_context)) + left_context)
            
            right_context = sent[ent2.end:ent2.end+3]
            rights.append(right_context + (['<END>']*(3-len(right_context))))
            
            middles.append(sent[ent1.end:ent2.start])
            
            ents1.append(ent1.string)
            ents2.append(ent2.string)
            
            tags.append([ent1.label_, ent2.label_])

            

In [ ]:
len(lefts)

Посмотрим какие вообще пары сущностей достались.

In [ ]:
entpair = ['#'.join(e) for e in zip(ents1, ents2)]

In [ ]:
Counter(entpair).most_common()

Почему-то выделяются новые строки. Но попробуем с ними.

In [ ]:
lefts_s = [' '.join([str(x) for x in l]) for l in lefts]
rights_s = [' '.join([str(x) for x in l]) for l in rights]
middles_s = [' '.join([str(x) for x in l]) for l in middles]


In [ ]:
len(lefts_s)

In [ ]:
# tfidf = TfidfVectorizer(max_features=3000)
# tfidf.fit(lefts_s + rights_s + middles_s)

In [ ]:
l = tfidf.transform(lefts_s)
r = tfidf.transform(rights_s)
m = tfidf.transform(middles_s)

In [ ]:
X_ = hstack([l,m,r])

In [ ]:
X_.shape

Теперь посмотрим на то, как предсказывается какой-нибудь класс.

In [ ]:
list(clf.classes_).index('P118')

In [ ]:
pred = clf.predict_proba(X_)[:, 5]

In [ ]:
list(zip(np.array(sents)[pred > 0.3], np.array(entpair)[pred > 0.3]))

Тема вроде бы распознается, но отношения выделяются не очень хорошо.

Можно попробовать кластеризовать предложения. Вдруг какие-то отношения выделятся в кластер.

In [ ]:
cluster = MiniBatchKMeans(100, verbose=1, reassignment_ratio=0.3, max_no_improvement=500)
cluster.fit(X_)

In [ ]:
cl2id = defaultdict(list)

for i, cl in enumerate(cluster.labels_):
    cl2id[cl].append(i)


In [ ]:
len(entpair)

In [ ]:
f = open('clusters.txt', 'w')

for cl in cl2id:
    f.write('CLUSTER __' + str(cl) + '__\n')
    for i in cl2id[cl]:
        f.write(entpair[i].replace('\n', ' ') + '\n')
        f.write(sents[i].replace('\n', ' ') + '\n')
    f.write('\n\n')
f.close()

## Другие подходы

Для нейросетей задачу можно представить как seq2seq - каждому токену соответствует тэг H, T или O (можно добавить Begin, Inside тэги, чтобы отметить многословные сущности). И можно вообще решать задачу извлечения сущностей и отношений вместе. Про это читать,
например, тут:  https://www.semanticscholar.org/paper/Joint-learning-of-named-entity-recognition-and-Xu-Li/31ce449618068343f9f83c904c7fd062ba943c8e?navId=references

Извлечение отношений часто пытаются решать без учителя. 

Один из подходов - **bootstrapping**, о котором мы говорили в прошлый раз. Можно выбрать какой-то набор пар сущностей, которые выражают отношение и найти предложения, в которых эта пара сущностей встречается. Потом найти похожие по контекстам предложения и считать их представителями этого класса. Затем можно достать пары сущностей, в которых они употреблены и повторить все заново. 

Про это можно почитать вот тут:

1) одна из первых работ - https://pdfs.semanticscholar.org/6f16/7cce628ec4983788ddf21587630afebf43ce.pdf?_ga=2.136426931.2051797770.1542970757-1216332217.1520769589 (от создателя гугла)

2) https://pdfs.semanticscholar.org/189e/d3f749766d02d42eb5b6d71017e085c212d4.pdf?_ga=2.112375175.2051797770.1542970757-1216332217.1520769589

3) Тут бустраппинг делается с помощью word2vec -https://pdfs.semanticscholar.org/fe6e/56ec0a1f5d673a4ab22e716f2c846b497f9c.pdf?_ga=2.179043751.2051797770.1542970757-1216332217.1520769589



Другой популярный метод - **distant supervision**. Идея очень похожая, только вместо того, чтобы самим придумывать положительные примеры - их берут из какой-нибудь базы данных. Например, из DBPedia или из Freebase. Достав большое количество упоминаний, можно собрать уже достаточно большую обучающую выборку.

Почитать можно тут:  
1) первая статья по теме (от журафского) https://www.semanticscholar.org/paper/Distant-supervision-for-relation-extraction-without-Mintz-Bills/8f8139b63a2fc0b3ae8413acaef47acd35a356e0  
    
2) тут предлагаются методы убрать шум из такой разметки - https://www.semanticscholar.org/paper/Denoising-Distant-Supervision-for-Relation-via-Han-Liu/3d13ee24493a6c2a0477b15e5145ba5868c3df40 
    
   

Ещё одно большое направление - Open Information Extraction. Идея тут в том, чтобы извлекать из предложений (или текстов) все отношения в виде троек (субъект, предикат, объект). 

Например, из предложения __The U.S. president Barack Obama gave his speech on Tuesday and Wednesday to thousands of people.__ излекутся тройки:  

__(Barack Obama, president, U.S)  
(Barack Obama, gave, his speech)  
(Barack Obama, gave his speech, on Tuesday)  
(Barack Obama, gave his speech, on Wednesday)  
(Barack Obama, gave his speech, to thousands of people)__

Таким образом можно извлекать неограниченное количество типов отношений, т.е. не нужно для каждого типа размечать предложения. Однако появляется необходимость каким-то образом кластеризовать разные способы выражения одного типа отношений. 

Почитать про OpenIE можно тут:

1) оригинальная работа http://www.aaai.org/Papers/IJCAI/2007/IJCAI07-429.pdf  
2) одна из самых известных работ http://ml.cs.washington.edu/www/media/papers/reverb_emnlp2011.pdf  
3) одна из последних статей http://www.cse.iitd.ac.in/~mausam/papers/coling18.pdf  

Реализация OpenIE есть StandfordNLP и вот тут - https://github.com/dair-iitd/OpenIE-standalone